In [ ]:
from keras.models import load_model # type: ignore
import tensorflow as tf # type: ignore
import numpy as np # type: ignore
from keras.preprocessing.image import ImageDataGenerator # type: ignore
from matplotlib import pyplot as plt # type: ignore
import cv2 # type: ignore
from keras.metrics import MeanSquaredError # type: ignore
from tensorflow import keras # type: ignore
import keras_tuner as kt # type: ignore

In [ ]:
# Charger le modèle VGG16 pré-entraîné
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet')

# Récupérer les noms des couches de convolution
layer_names = [layer.name for layer in vgg.layers if 'conv' in layer.name]

# Créer un dictionnaire pour stocker les modèles des couches de convolution
conv_layers = {}

# Stocker chacune des couches dans le dictionnaire
for i, layer_name in enumerate(layer_names):
    conv_layers[f'get_features_maps_{i}'] = tf.keras.Model(inputs=vgg.input, outputs=vgg.get_layer(layer_name).output)

def normalize(x):
    # Fonction min-max pour les Tensors
    min_val = tf.reduce_min(x)
    max_val = tf.reduce_max(x)
    return (x - min_val) / (max_val - min_val)

def lpips(batch_pred, batch_true):
    # Convertir les images en 3 canaux parce qu'elles sont en niveaux de gris
    batch_pred = tf.image.grayscale_to_rgb(batch_pred)
    batch_true = tf.image.grayscale_to_rgb(batch_true)

    # Appliquer le prétraitement VGG16 aux images
    batch_pred = tf.keras.applications.vgg16.preprocess_input(batch_pred)
    batch_true = tf.keras.applications.vgg16.preprocess_input(batch_true)

    # Initialiser la perte totale
    total_loss = 0.0

    # Extraire et normaliser les cartes de caractéristiques des deux images pour chaque couche de convolution
    for key in conv_layers.keys():
        pred_features = conv_layers[key](batch_pred)
        true_features = conv_layers[key](batch_true)
        total_loss += tf.reduce_mean(tf.square(pred_features - true_features))

    # Normaliser la perte totale
    total_loss /= len(conv_layers)

    """ # Ajouter la différence au niveau pixel
    pixel_diff = tf.reduce_mean(tf.square(batch_pred - batch_true))
    total_loss += pixel_diff """

    return total_loss

In [ ]:
def residual_block(x, filters, kernel_size=3, stride=1, activate=True):
    shortcut = x
    # Convolutions
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=1, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        # Ajuste les dimensions
        shortcut = tf.keras.layers.Conv2D(filters, 1, strides=stride, padding='same', use_bias=False)(shortcut)
        shortcut = tf.keras.layers.BatchNormalization()(shortcut)
    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x

def residual_block_recon(x, filters, kernel_size=3, stride=1, activate=True):
    shortcut = x
    # Première convolution
    x = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Deuxième convolution
    x = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=1, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Ajuste les dimensions
    shortcut = tf.keras.layers.Conv2DTranspose(filters, 1, strides=stride, padding='same', use_bias=False)(shortcut)
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)

    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def generator_to_array(generator):
    # Initialiser une liste pour stocker les échantillons
    num_samples = len(generator)
    samples = []
    # Itérer sur le générateur pour obtenir les échantillons
    for i in range(num_samples):
        batch = generator.next()
        for image in batch[0]:
            samples.append(image)  # Ajouter uniquement les données (ignorer les étiquettes)
    return np.array(samples)

def comparaison_visages(asian, asian_predict, white, white_predict, ethnie):

    num_images = 6
    plt.figure(figsize=(10, 4))
    for i in range(num_images):
        # Afficher l'image originale
        ax = plt.subplot(2, num_images, i + 1)
        plt.imshow(cv2.resize(white[i],(224, 224), interpolation=cv2.INTER_LINEAR), cmap='gray')
        plt.title("Original")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax = plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(cv2.resize(white_predict[i], (224, 224), interpolation=cv2.INTER_LINEAR), cmap='gray')
        plt.title("Reconstruction")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
    
    plt.figure(figsize=(10, 4))
    for i in range(num_images):
        # Afficher l'image originale
        ax = plt.subplot(2, num_images, i + 1)
        plt.imshow(cv2.resize(asian[i],(224, 224), interpolation=cv2.INTER_LINEAR), cmap='gray')
        plt.title("Original")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax = plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(cv2.resize(asian_predict[i], (224, 224), interpolation=cv2.INTER_LINEAR), cmap='gray')
        plt.title("Reconstruction")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
trainset_white = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Train',
    classes=['Caucasiens'],
    target_size=(224, 224), 
    batch_size=4, 
    class_mode='input',
    color_mode='grayscale')
testset_white = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Test',
    classes=['Caucasiens'],
    target_size=(224, 224), 
    batch_size=4, 
    class_mode='input',
    color_mode='grayscale')
trainset_asian = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Train',
    classes=['Asiatiques'],
    target_size=(224, 224),
    batch_size=4,
    class_mode='input',
    color_mode='grayscale')
testset_asian = train_datagen.flow_from_directory(
    '../../Datasets/STIM_NB_LumNorm/Test',
    classes=['Asiatiques'],
    target_size=(224, 224),
    batch_size=4,
    class_mode='input',
    color_mode='grayscale')

In [ ]:
white = generator_to_array(trainset_white)
var = 0
for i in range(50):
    var += white[i].var()
var

In [ ]:
asian = generator_to_array(trainset_asian)
var = 0
for i in range(50):
    var += asian[i].var()
var

In [ ]:
def build_model(hp, shape=(224, 224, 1), input_latent=64):
    input_img = tf.keras.Input(shape=shape)
    x = tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', use_bias=False)(input_img)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D(2, strides=2, padding='same')(x)

    # Encodeur
    num_blocks = hp.Int('num_residual_blocks', min_value=3, max_value=7, step=2)
    filters = [64, 128, 256, 512, 1024, 2048, 250]
    strides = [2, 2, 2, 2, 2, 2, 1]
    for i in range(num_blocks):
        x = residual_block(x, filters[i % len(filters)], 3, strides[i % len(strides)])

    x = tf.keras.layers.Flatten()(x)
    latent_space_layer = tf.keras.layers.Dense(input_latent, activation='relu', use_bias=False)(x)
    latent_space_layer_norm = tf.keras.layers.BatchNormalization(name='latent_space_layer_norm')(latent_space_layer)

    # Décodeur
    reshape_layer = tf.keras.layers.Reshape(target_shape=(1, 1, input_latent))(latent_space_layer_norm)
    filters_decoder = filters[:num_blocks][::-1]  # Inverser les filtres utilisés dans l'encodeur
    strides_decoder = strides[:num_blocks][::-1]  # Inverser les strides utilisés dans l'encodeur

    x_recon = reshape_layer
    for i in range(num_blocks):
        x_recon = residual_block_recon(x_recon, filters_decoder[i], 3, strides_decoder[i])

    x_recon = tf.keras.layers.Conv2DTranspose(32, 3, strides=2, padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.BatchNormalization()(x_recon)
    x_recon = tf.keras.layers.Activation('relu')(x_recon)
    x_recon = tf.keras.layers.UpSampling2D(size=(2, 2))(x_recon)

    x_recon = tf.keras.layers.Conv2DTranspose(1, 1, activation='sigmoid', padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.Resizing(224, 224)(x_recon)
    model = tf.keras.Model(inputs=input_img, outputs=[x_recon])

    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    if optimizer_choice == 'adam':
        optimizer = keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        )
    elif optimizer_choice == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        )
    elif optimizer_choice == 'sgd':
        optimizer = keras.optimizers.SGD(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        )
    model.compile(optimizer=optimizer, loss=lpips, metrics=MeanSquaredError())
    #model.summary()
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective('val_MeanSquaredError', direction='min'),  # Objectif de minimiser la métrique SSIM
    max_trials=20,  # Ajustez ce nombre en fonction de vos besoins
    executions_per_trial=1,  # Nombre d'exécutions par essai pour obtenir une moyenne
    directory='my_dir',
    project_name='residual_blocks_lpips'
)

In [ ]:
tuner.search(trainset_white, epochs=10, validation_data=testset_white)

In [ ]:
tuner.get_best_hyperparameters()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

In [ ]:
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(trainset_white, epochs=1000, validation_data=testset_white)
best_model.save('Modeles/STIM/best_model_white.h5')

In [ ]:
test_asian = generator_to_array(testset_asian).reshape(6, 224, 224)
test_asian_predict = best_model.predict(testset_asian).reshape(6, 224, 224)
test_white = generator_to_array(testset_white).reshape(6, 224, 224)
test_white_predict = best_model.predict(testset_white).reshape(6, 224, 224)
comparaison_visages(test_asian, test_asian_predict, test_white, test_white_predict, "white")

In [ ]:
best_model = tuner.hypermodel.build(best_hps)
history2 = best_model.fit(trainset_asian, epochs=1000, validation_data=testset_asian)
best_model.save('Modeles/STIM/best_model_asian.h5')

In [ ]:
test_asian = generator_to_array(testset_asian).reshape(6, 224, 224)
test_asian_predict = model.predict(testset_asian).reshape(6, 224, 224)
test_white = generator_to_array(testset_white).reshape(6, 224, 224)
test_white_predict = model.predict(testset_white).reshape(6, 224, 224)
comparaison_visages(test_asian, test_asian_predict, test_white, test_white_predict, "white")